# Section 4.2: Currying: a different way to look at functions
## * Named after Haskell Curry, a mathematician and logician who perfected this concept from the ideas first introduced by Gottlob Frege and Moses Schönfinkel
## * Alternative to "partial function application" covered in the previous section
## * Create unary functions that return reference to other unary functions for each parameter required by the higher level function.

In [2]:
// greater : (double, double) → bool
bool greater(double first, double second)
{
    return first > second;
}

In [3]:
// greater_curried : double → (double → bool)
auto greater_curried(double first)
{    
    return [first](double second) { return first > second; };
}

In [4]:
// Invocation:
#include <iostream>

// Returns false
std::cout << greater(2, 3) << std::endl;

0


In [7]:
// Returns a unary function object that checks whether its argument is less than 2
greater_curried(2);

In [8]:
// Returns false
std::cout << greater_curried(2)(3) << std::endl;

0


![Figure 4.9](figure_4.9.png)

In [ ]:
auto print_person_cd(const person_t& person)
{    
    return [&](std::ostream& out) 
    {        
        return [&](person_t::output_format_t format) 
        {            
            print_person(person, out, format);        
        };    
    };
}

![Figure 4.10](figure_4.10.png)

# Writing code like this is tedious.
## * Use a helper function to convert any function into it's curried version.


In [ ]:
using std::cout;

auto print_person_cd = make_curried(print_person);

// All of these write Martha's name and surname to the standard output.
print_person_cd(martha, cout, person_t::full_name); 
print_person_cd(martha)(cout, person_t::full_name);
print_person_cd(martha, cout)(person_t::full_name);
print_person_cd(martha)(cout)(person_t::full_name);

// Returns a function object that will *only* print Martha's name to standard output.
auto print_martha = print_person_cd(martha);
print_martha(cout, person_t::name_only);

// Returns a function object that will print some amount of Martha's information to standard output.
auto print_martha_to_cout = print_person_cd(martha, cout);
print_martha_to_cout(person_t::name_only);

# Considering a real world example
## * Using currying with database access
## * With currying it's possible to create funtions that are composable without overloading

In [ ]:
auto table  = "Movies";
auto filter = "Name = \"Sintel\"";

// Uses the query function as a normal function by passing all required 
// arguments to it and getting a list of query results directly
results = query(local_connection, session,
                            table, filter);

// Creates separate functions that are tied to their respective database 
// connections. This is useful when you often use the same connection 
// over and over.
auto local_query = query(local_connection);
auto remote_query = query(remote_connection);

    ...
    
results = local_query(session, table, filter);

// When you need only a single connection and a single session, you can 
// create a function that binds these two values so that you can omit 
// them in the rest of the program.
auto main_query = query(local_connection,
                            main_session);
   ...
    
results = main_query(table, filter);

// If you often perform queries over the same table, 
// you can create a function that always queries that table
auto movies_query = main_query(table);

    ... 
        
results = movies_query(filter);

## * Assume that make_curried() has been used to create the above functions
## * Users can now create the functions they need
## * A single implementation of the function (now curried) can provide all of the functional variations
## * The above statement assumes that the parameters ordering does not vary

# Comparing std::bind() to currying

In [ ]:
// Binds only the connection argument to the local query.
auto local_query = query(local_connection);

// You can bind the first argument, but you aren’t required to. You can bind just the second one.
auto local_query = std::bind(query, local_connection, _1, _2, _3);
auto session_query = std::bind(query, _1, main_session, _2, _3);

# std::bind() is more flexible but requires that all arguments be accounted for

In [ ]:
// With currying, however, you simply create a new function object with the required parameters.
auto local_query  = query(local_connection);
auto local_update = update(local_connection);

In [ ]:
// With std::bind() you would need to create a function template for the local connection.
template <typename Function> 
auto for_local_connection(Function f) 
{    
    return f(local_connection);
}

auto local_query = for_local_connection(query);
auto local_update = for_local_connection(update);
auto local_delete = for_local_connection(delete);

## * Partial function application is useful when a function is needed with specific arguments to bind.
## * Currying is useful in more generic contexts with any number of arguments.

# 4.3 Demonstrating functional composibility

In [ ]:
tr -cs A-Za-z '\n' |    
    tr A-Z a-z |    
    sort |    
    uniq -c |    
    sort -rn |    
    sed ${1}q

![Figure 4.12](figure_4.12.png)

## * Create five functions:

In [ ]:
// Receive a string and return a collection of words: std::vector<std::string>
std::vector<std::string> words(const std::string& text);

In [ ]:
// Receive a collection of words, and return a map of the words and the number of times it appears in the text.
template <typename T>
std::unordered_map<T, unsigned int> count_occurrences(
    const std::vector<T>& items);

// The type of the contained items could also be template-ized:
template <typename C, typename T = typename C::value_type>
std::unordered_map<T, unsigned int> count_occurrences(
    const C& collection);

In [ ]:
// Takes each pair of values from the container and creates a new pair with the values reversed
// to that it can be sorted later. Values are returned as a vector.
template <typename C,
          typename P1,
          typename P2>
std::vector<std::pair<P2, P1>> reverse_pairs(
        const C& collection);

In [ ]:
// Sort the vector by frequency and print to the standard output, combining with the previous functions.
void print_common_words(const std::string& text)
{    
    return print_pairs(
        sort_by_frequency(
            reverse_pairs(
                count_occurrences(
                    words(text)
                )
            )
        )
    );
}

# Demonstrates how code written in the imperative style can be split into a few simple functions.

# 4.4 Function lifting, revisited

In [ ]:
// Collection of functions that operate on strings:

void pointer_to_upper(std::string* str)    ①  
{    ①      
    if (str) to_upper(*str);    ①  
}    ①  

void vector_to_upper(std::vector<std::string>& strs)    ②  
{    ②      
    for (auto& str : strs) {    ②          
        to_upper(str);    ②      
    }    ②  
}    ②  

void map_to_upper(std::map<int, std::string>& strs)    ③  
{    ③      
    for (auto& pair : strs) {    ③
        to_upper(pair.second);    ③      
    }    ③  
}

# Function Lifting: "Lifts" the function that operates on a particular type to a struture or a collection containing the same type

In [ ]:
// Using auto as the type specifier to allow the function to be used with pointers to 
// any type, not just strings.
template <typename Function> 
auto pointer_lift(Function f)
{    
    return [f](auto* item) {    ①
        if (item) {    ①
            f(*item);    ①
        }    ①
                           };    ①
}

// This function allows collection references of any type to be used.
template <typename Function>
auto collection_lift(Function f)
{    
    return [f](auto& items) {    ②
        for (auto& item : items) {    ②
            f(item);    ②          
        }    ②
    };    ②
}

# This technique can be applied to any type, allowing the simplest case possible to be implemented and then lifted to the structure where it will be used.
# Lifting a lambda:

In [ ]:
// Initializes type C to be the type of the collection, P1 to be 
// the type of the first item in a pair coming from the source 
// collection (with const removed), and P2 to be the type of 
// second item in the pair
template <    
    typename C,    ①
     typename P1 = typename std::remove_cv<    ①
        typename C::value_type::first_type>::type,    ①
     typename P2 = typename C::value_type::second_type    ①
    >
    
std::vector<std::pair<P2, P1>> reverse_pairs(const C& items)
{    
    std::vector<std::pair<P2, P1>> result(items.size());

    // Passes a lambda that reverses values in a single pair to 
    // std::transform, and lifts it to a new function that can perform 
    // the same task but on multiple items in a collection
    std::transform(    ②
        std::begin(items), std::end(items),    ②
        std::begin(result),    ② 
        [](const std::pair<const P1, P2>& p)    ②   
        {    ②   
            return std::make_pair(p.second, p.first);   ② 
        }    ②      
    );    
    
    return result;
    
}

# Lifts a function that takes a pair of values of a particular type, to work on an arbitrary collection of pairs.
# Lifting allows you to easily implement these functions when they just need to pass the torch on to a function that operates on the underlying type.

## Time permitting: Using the Xeus-Cling Jupyter docker image and mybinder.org

### https://github.com/airbrain-org/xeus-cling/blob/master/README2.md

### https://mybinder.org
### https://mybinder.org/v2/gh/QuantStack/xeus-cling/stable?filepath=notebooks/xcpp.ipynb